# Lab 1: Persistent Storage Basics

This workshop assumes a basic knowledge of Kubernetes and some familiarity with persistent storage. You should've received a briefing about the core concepts and an introductory presentation to the workshop nomenclatures we're using prior to accessing the labs.

## Get started!

In this workshop, we'll utilize the "hpe-standard" `StorageClass`. It uses the HPE CSI Driver. It's common in Kubernetes application deployment to not have more than one `StorageClass` and for that `StorageClass` to be marked "default". That way users don't have to call out which `StorageClass` to use. Since the HPE Data Fabric is marked "default" on the HPE Ezmeral Container Platform, we need to be explicit. 

Let's inspect the `StorageClasses` available on the cluster.

In [ ]:
kubectl get storageclasses

Examine the "hpe-standard" `StorageClass` explicitly.

In [ ]:
kubectl get storageclasses/hpe-standard -o yaml

The parameters prefixed with "csi.storage.k8s.io" are needed for the CSI sidecars to communicate with the HPE CSI Driver, both for the Controller and Node driver. The remaining parameters are specific to the HPE CSI Driver *or* the backend CSP. If you want to study which parameters goes to the HPE CSI Driver and what parameters are available for each of the supported backends, check out these resources on the HPE Storage Container Orchestrator Documentation (SCOD) portal.

- [Base `StorageClass` parameters for the HPE CSI Driver](https://scod.hpedev.io/csi_driver/using.html#base_storageclass_parameters)
- [HPE Nimble Storage parameters](https://scod.hpedev.io/container_storage_provider/hpe_nimble_storage/index.html#storageclass_parameters)
- [HPE Primera and HPE 3PAR parameters](https://scod.hpedev.io/container_storage_provider/hpe_3par_primera/index.html#storageclass_parameters)

Other `StorageClass` attributes that are of interest from an end-user perspective is the `.reclaimPolicy` (what happens to the `PersistentVolume` when the `PersistentVolumeClaim` is deleted) and `.allowVolumeExpansion` (if the `StorageClass` allows volume expansion or not).

## Your first claim

Review the `PersistentVolumeClaim` below.

---
```yaml
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-first-pvc
spec:
  accessModes:
  - ReadWriteOnce
  resources:
    requests:
      storage: 128Mi
  storageClassName: hpe-standard
```
---

In the `.spec`, the `.spec.accessModes` and `.spec.resources.requests.storage` are mandatory. `.spec.storageClassName` is necessary in our case as we specifically want to use the "hpe-standard" `StorageClass`. The different access modes are discussed later in the workshop. We'll use `ReadWriteOnce` until then.

In [ ]:
kubectl create -f obj/my-first-pvc.yaml

Let's inspect our newly created PVC.

In [ ]:
kubectl get persistentvolumeclaim/my-first-pvc -o yaml

The important aspects of the claim is the `.spec.status.phase` where it should say "Bound". It may be say "Pending" if the cluster or backend is busy. Rerun the previous command to ensure the phase says "Bound" before proceeding.

It's also possible to observe the backing `PersistentVolume` in `.spec.volumeName`. However, since `PersistentVolumes` are cluster-wide objects, restricted users are only allowed to observe the name by reference through the `PersistentVolumeClaim`.

Also, pay attention to `.spec.volumeMode`, which by default is set to "Filesystem". Later in the lab, we'll create a `PersistentVolumeClaim` where we request "Block" and show you how to access the raw block device from a `Pod`.

## Attach a PersistentVolumeClaim to a workload

We should now have a `PersistentVolumeClaim` named "my-first-pvc". To attach it to a workload, it needs to be referenced in the `.spec.volumes` stanza of the controller. Let's create a `Deployment` referencing our claim.

---
```yaml
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-first-deploy
  labels:
    app: my-first-deploy
spec:
  replicas: 1
  strategy:
    type: Recreate
  selector:
    matchLabels:
      app: my-ioping
  template:
    metadata:
      labels:
        app: my-ioping
    spec:
      containers:
      - image: hpestorage/ioping
        name: ioping
        command: [ "ioping" ]
        args: [ "/data" ]
        volumeMounts:
          - name: my-data
            mountPath: /data
      volumes:
      - name: my-data
        persistentVolumeClaim:
          claimName: my-first-pvc
```
---

In [ ]:
kubectl create -f obj/my-first-deploy.yaml
kubectl rollout status deploy/my-first-deploy

It should take a few seconds or so before the `Deployment` comes up. Run the below command when it says `1/1` in the "READY" column.

In [ ]:
kubectl get deployments/my-first-deploy -o wide

The `Pod` we declared in the `Deployment` sends a 4KiB IO to the specified mountpoint and sends the results to `stdout`. To verify the `Pod` is indeed working, run the following:

In [ ]:
kubectl logs --since=5s deployments/my-first-deploy

We should be able to observe a 4KiB IO sent to the `/data` mountpoint that hosts a XFS filesystem on a multipath device.

Leave the `Deployment` running as we go through the next exercise.

## Resize a PersistentVolumeClaim

Resizing a `PersistentVolumeClaim` is done with either the `kubectl edit` or `kubectl patch` sub-command. Since we're using Jupyter notebooks, "patch" is more practical. Let's inspect the current `PersistentVolumeClaim` size from the workload's perspective.

In [ ]:
kubectl exec -it deployment/my-first-deploy -- df -h /data

Let's double the volume size.

In [ ]:
kubectl patch persistentvolumeclaim my-first-pvc -p '{"spec":{"resources":{"requests":{"storage": "256Mi"}}}}'

Now, wait a couple of minutes (skip to the next exercise and come back). Then re-run the `df` command.

In [ ]:
kubectl exec -it deployment/my-first-deploy -- df -h /data

The new size should be reflected on the device. How does it compare to the previous output?

# Working with Raw Block Devices

CSI is capable of exposing a representation of the underlying block device to the `Pod`. By default, the `.spec.volumeMode` of a `PersistentVolumeClaim` is set to "Filesystem". Let's create a block-based `PersistentVolumeClaim`.

---
```yaml
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-block-pvc
spec:
  accessModes:
  - ReadWriteOnce
  resources:
    requests:
      storage: 128Mi
  storageClassName: hpe-standard
  volumeMode: Block
```
---

In [ ]:
kubectl create -f obj/my-block-pvc.yaml

Create a `Deployment` that accesses the block-based `PersistentVolumeClaim`.

---
```yaml
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-block-deploy
  labels:
    app: my-block-deploy
spec:
  replicas: 1
  strategy:
    type: Recreate
  selector:
    matchLabels:
      app: my-block-ioping
  template:
    metadata:
      labels:
        app: my-block-ioping
    spec:
      containers:
      - image: hpestorage/ioping
        name: ioping
        command: [ "ioping" ]
        args: [ "/dev/xvda" ]
        volumeDevices:
          - name: my-data
            devicePath: /dev/xvda
      volumes:
      - name: my-data
        persistentVolumeClaim:
          claimName: my-block-pvc
```
---

Pay attention to a few key differences versus using `.spec.volumeMode: Filesystem` in the `PersistentVolumeClaim`. In the `Deployment` stanza, `.spec.template.spec.containers`, you'll see `volumeDevices` and `devicePath` instead of `volumeMounts` and `mountPath`.

Go ahead and create the deployment.

In [ ]:
kubectl create -f obj/my-block-deploy.yaml
kubectl rollout status deploy/my-block-deploy

When the `Pod` comes up, check the logs.

In [ ]:
kubectl logs --since=5s deployments/my-block-deploy

The IO is now being issues against the block device exposed to the `Pod` instead of the filesystem.

**Question:** Is the IO faster, slower or similar to the one issued in the case of `volumeMode: Filesystem`?

## Cleaning up!

We'll clean up after you eventually, but if you would be so kind as to free up some resources before we skip to the next set of exercises, it would be greatly appreciated.

In [ ]:
kubectl delete deploy/my-first-deploy deploy/my-block-deploy pvc/my-first-pvc pvc/my-block-pvc

Let's continue on to [Lab 2: Snapshots and Clones](2-WKSHP-CSI-DataManagement.ipynb)

> **Learn more**
> - {{ BRANDING }} Community Blog: [Introducing a multi-vendor CSI driver for Kubernetes](https://developer.hpe.com/blog/n0J8kpk1DJf4y7xD2D4X/introducing-a-multi-vendor-csi-driver-for-kubernetes)